In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
import re

In [3]:
df = pd.read_excel("Concat.xlsx")
df.head()

,Unnamed: 0,Sector Name,Sub Sector,Organization,Operation,Operation Type,Address,City,Postal Code,Annual Flow,...,Fuel Oil 4 & 6 Qty ltr,Propane Qty ltr,Coal Qty metric ton,Wood Qty metric ton,District Heating Qty gj,District Cooling Qty gj,GHG Emissions metric ton,year,Fuel Oil 4 Qty ltr,Fuel Oil 5 & 6 Qty ltr
0,0,Public Hospital,Acute/Chronic,Alexandra Hospital,Alexandra Hospital,Facilities used for hospital purposes,29 Noxon Street,Ingersoll,N5C3V6,0.0,...,0.0,0.0,0,0.0,0.0,0.0,528.0,2012,NaN,NaN
1,1,Public Hospital,Acute/Chronic,Alexandra Hospital,Alexandra Hospital,Administrative offices and related facilities,29 Noxon Street,Ingersoll,N5C3V6,0.0,...,0.0,0.0,0,0.0,0.0,0.0,432.0,2012,NaN,NaN
2,2,Public Hospital,Acute/Chronic,Alexandria Glengarry Memorial Hospital,Glengarry Memorial Hospital,Facilities used for hospital purposes,20260 County Road 43,Alexandria,K0C1A0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,587.0,2012,NaN,NaN
3,3,Public Hospital,Acute/Chronic,Alexandria Glengarry Memorial Hospital,Glengarry Memorial Hospital,Administrative offices and related facilities,20260 County Road 43,Alexandria,K0C1A0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,45.0,2012,NaN,NaN
4,4,School Board,School Board,Algoma District School Board,Adult Ed - Blind River (WC Eaket),Schools,147 Woodward Ave,Blind River,P0R1B0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,53.0,2012,NaN,NaN


In [4]:
df.isnull().sum()

Unnamed: 0                       0
Sector Name                      0
Sub Sector                       5
Organization                     0
Operation                        0
Operation Type                   0
Address                        107
City                             1
Postal Code                    180
Annual Flow                  25791
Electricity Qty kWh           1651
NaturalGas Qty m3             6205
Fuel Oil 1 & 2 Qty ltr       15559
Fuel Oil 4 & 6 Qty ltr       15889
Propane Qty ltr              15129
Coal Qty metric ton          15893
Wood Qty metric ton          15892
District Heating Qty gj      15858
District Cooling Qty gj      15861
GHG Emissions metric ton      1786
year                             0
Fuel Oil 4 Qty ltr          170310
Fuel Oil 5 & 6 Qty ltr      170328
dtype: int64

In [90]:
df.drop(columns=['Unnamed: 0'], inplace=True)

### Here we are treating the null value

In [5]:
df['Sub Sector'] = df.apply(lambda row: row['Organization'].split()[1] if pd.isnull(row['Sub Sector']) else row['Sub Sector'], axis=1)

In [6]:
df[(df['Address'] == '-') | (df['Address'] == '0') | (df['Address'] == '1')]
df['Address'].replace({'-': np.nan, '0': np.nan, '1': np.nan}, inplace=True)
df['Address'].ffill(inplace=True)

In [7]:
df[(df['City'] == '-') | (df['City'] == '0') | (df['City'] == '1')]
df['City'].replace({'-': np.nan, '0': np.nan, '1': np.nan}, inplace=True)
df['City'].ffill(inplace=True)

In [8]:
df[(df['Postal Code'] == '-') | (df['Postal Code'] == '0') | (df['Postal Code'] == '1')]
df['City'].replace({'-': np.nan, '0': np.nan, '1': np.nan}, inplace=True)
df['Postal Code'].ffill(inplace=True)

In [9]:
df.drop(columns=['Annual Flow'], inplace=True)

In [62]:
df['City'] = df['City'].apply(lambda x: x.title().strip())

In [71]:
df.loc[165389, 'City'] = 'Oro Medonte'

In [72]:
df['Electricity Qty kWh'].isnull().sum()

1651

In [73]:
city_means = df.groupby('City')['Electricity Qty kWh'].mean()

for city in df['City'].unique():
    city_mask = df['City'] == city
    df.loc[city_mask, 'Electricity Qty kWh'] = df.loc[city_mask, 'Electricity Qty kWh'].fillna(city_means[city])

In [76]:
df['Electricity Qty kWh'].isnull().sum()

0

In [75]:
df['Electricity Qty kWh'].fillna(df['Electricity Qty kWh'].median(), inplace=True)

In [77]:
city_means = df.groupby('City')['NaturalGas Qty m3'].median()

for city in df['City'].unique():
    city_mask = df['City'] == city
    df.loc[city_mask, 'NaturalGas Qty m3'] = df.loc[city_mask, 'NaturalGas Qty m3'].fillna(city_means[city])

In [83]:
df['NaturalGas Qty m3'].isnull().sum()

0

In [82]:
df['City'][df['NaturalGas Qty m3'].isnull()]

Series([], Name: City, dtype: object)

In [81]:
df['NaturalGas Qty m3'].fillna(df['NaturalGas Qty m3'].median(), inplace=True)

In [84]:
df['Fuel Oil 4 Qty ltr'].fillna(0, inplace=True)
df['Fuel Oil 5 & 6 Qty ltr'].fillna(0, inplace=True)

In [92]:
l = list(df.columns)
l

['Sector Name',
 'Sub Sector',
 'Organization',
 'Operation',
 'Operation Type',
 'Address',
 'City',
 'Postal Code',
 'Electricity Qty kWh',
 'NaturalGas Qty m3',
 'Fuel Oil 1 & 2 Qty ltr',
 'Fuel Oil 4 & 6 Qty ltr',
 'Propane Qty ltr',
 'Coal Qty metric ton',
 'Wood Qty metric ton',
 'District Heating Qty gj',
 'District Cooling Qty gj',
 'GHG Emissions metric ton',
 'year',
 'Fuel Oil 4 Qty ltr',
 'Fuel Oil 5 & 6 Qty ltr']

In [93]:
l.index('Fuel Oil 1 & 2 Qty ltr')

10

In [94]:
l.index('year')

18

In [95]:
l[10:18]

['Fuel Oil 1 & 2 Qty ltr',
 'Fuel Oil 4 & 6 Qty ltr',
 'Propane Qty ltr',
 'Coal Qty metric ton',
 'Wood Qty metric ton',
 'District Heating Qty gj',
 'District Cooling Qty gj',
 'GHG Emissions metric ton']

In [98]:
df['Coal Qty metric ton'] = df['Coal Qty metric ton'].apply(lambda x: x if x != 'Litre' else np.nan)

In [99]:
df['Coal Qty metric ton'][df['Coal Qty metric ton']=='Litre']

Series([], Name: Coal Qty metric ton, dtype: float64)

In [100]:
for col in l[10:18]:
    city_means = df.groupby('City')[col].median()

    for city in df['City'].unique():
        city_mask = df['City'] == city
        df.loc[city_mask, col] = df.loc[city_mask, col].fillna(city_means[city])
        df[col].fillna(df[col].median(), inplace=True)

In [101]:
df.isnull().sum()

Sector Name                 0
Sub Sector                  0
Organization                0
Operation                   0
Operation Type              0
Address                     0
City                        0
Postal Code                 0
Electricity Qty kWh         0
NaturalGas Qty m3           0
Fuel Oil 1 & 2 Qty ltr      0
Fuel Oil 4 & 6 Qty ltr      0
Propane Qty ltr             0
Coal Qty metric ton         0
Wood Qty metric ton         0
District Heating Qty gj     0
District Cooling Qty gj     0
GHG Emissions metric ton    0
year                        0
Fuel Oil 4 Qty ltr          0
Fuel Oil 5 & 6 Qty ltr      0
dtype: int64

In [102]:
df.to_excel('Final_Concated.xlsx')